### colab 한글
- 메뉴에서 런타임 -> 런타임 다시 시작


In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (7,823 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

###구글 드라이브 연동

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/colab/dataanalysis

Mounted at /content/drive
/content/drive/MyDrive/colab/dataanalysis


In [ ]:
!pip install folium
!pip install missingno

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import missingno as msno
import folium

In [6]:
# colab
plt.rc('font', family='NanumBarunGothic')

In [7]:
kmovies = pd.DataFrame({'MovieID':['1193','1194','1195'],
                        'Title':['디바','무림의 고수','정직한 후보'],
                        'Genres':['드라마','액션','코미디']})


kusers= pd.DataFrame({'UserID':[1,2,3,4,5],
                      'Gender':['F','M','M','F','M'],
                      'Age':[13,56,25,33,43]})


kratings= pd.DataFrame({'UserID':[1,1,1,2,2,2,3,3,3,4,4,4],
                        'MovieID':['1193','1194','1195','1193','1194','1195','1193','1194','1195','1193','1194','1195'],
                        'Rating':[2,5,None,3,1,5,None,2,5,5,None,3]})


In [8]:
print(type(kmovies))

<class 'pandas.core.frame.DataFrame'>


In [9]:
kmovies

,MovieID,Title,Genres
0,1193,디바,드라마
1,1194,무림의 고수,액션
2,1195,정직한 후보,코미디


In [10]:
kusers

,UserID,Gender,Age
0,1,F,13
1,2,M,56
2,3,M,25
3,4,F,33
4,5,M,43


In [13]:
kratings

,UserID,MovieID,Rating
0,1,1193,2.0
1,1,1194,5.0
2,1,1195,NaN
3,2,1193,3.0
4,2,1194,1.0
5,2,1195,5.0
6,3,1193,NaN
7,3,1194,2.0
8,3,1195,5.0
9,4,1193,5.0


In [14]:
kdata = kratings.merge(kusers)
kdata

,UserID,MovieID,Rating,Gender,Age
0,1,1193,2.0,F,13
1,1,1194,5.0,F,13
2,1,1195,NaN,F,13
3,2,1193,3.0,M,56
4,2,1194,1.0,M,56
5,2,1195,5.0,M,56
6,3,1193,NaN,M,25
7,3,1194,2.0,M,25
8,3,1195,5.0,M,25
9,4,1193,5.0,F,33


In [15]:
kdata = kratings.merge(kusers).merge(kmovies)
kdata

,UserID,MovieID,Rating,Gender,Age,Title,Genres
0,1,1193,2.0,F,13,디바,드라마
1,2,1193,3.0,M,56,디바,드라마
2,3,1193,NaN,M,25,디바,드라마
3,4,1193,5.0,F,33,디바,드라마
4,1,1194,5.0,F,13,무림의 고수,액션
5,2,1194,1.0,M,56,무림의 고수,액션
6,3,1194,2.0,M,25,무림의 고수,액션
7,4,1194,NaN,F,33,무림의 고수,액션
8,1,1195,NaN,F,13,정직한 후보,코미디
9,2,1195,5.0,M,56,정직한 후보,코미디


In [17]:
krecom_data = kdata[['UserID','MovieID','Rating']].copy()
krecom_data

,UserID,MovieID,Rating
0,1,1193,2.0
1,2,1193,3.0
2,3,1193,NaN
3,4,1193,5.0
4,1,1194,5.0
5,2,1194,1.0
6,3,1194,2.0
7,4,1194,NaN
8,1,1195,NaN
9,2,1195,5.0


### pivot
- 각 컬럼의 값을 교차테이블 구성요소로 전달, 교차테이블 완성  
- index, columns, values 컬럼을 각각 전달  
- 요약기능 없음  
- dataframe.pivot(index, columns, values)  

In [18]:
krecom_pivot = krecom_data.pivot(index='UserID', columns='MovieID', values='Rating')
krecom_pivot

MovieID,1193,1194,1195
UserID,,,
1,2.0,5.0,NaN
2,3.0,1.0,5.0
3,NaN,2.0,5.0
4,5.0,NaN,3.0


In [19]:
#inplace = True는 실제 데이터에 변경사항이 적용된다.
# krecom_pivot.fillna(method='ffill') #  method='ffill' or method='pad # 결측값을 앞방향으로 채운다.
# krecom_pivot.fillna(method='bfill') # method='bfill' or method='backfill' #결측값을 뒤방향으로 채운다.
# krecom_pivot.fillna(krecom_pivot.mean(), axis=1) # 현재 컬럼에 있는 값의 평균을 구하라

krecom_pivot.fillna(0, inplace=True) # NaN 값이 0으로 들어가는건가?... 결측값을 0으로 채워라

In [20]:
krecom_pivot

MovieID,1193,1194,1195
UserID,,,
1,2.0,5.0,0.0
2,3.0,1.0,5.0
3,0.0,2.0,5.0
4,5.0,0.0,3.0


### pivot_table
- pivot 기능과 유사, 더 많은 옵션 사용가능
- index, columns, values 컬럼을 각각 전달
- aggfunc 옵션 사용하여 요약 기능 전달 가능(기본은 평균)
- fill_value 옵션 사용하여 NA값 대체 가능
- datafame.pivot_table(index, columns, calues, aggfunc='mean', fill_value)  

In [21]:
krecom_pivot_table = krecom_data.pivot_table(index='UserID', columns='MovieID', values='Rating', aggfunc='min', fill_value=0)
krecom_pivot_table # 정수값으로 리턴된다.

MovieID,1193,1194,1195
UserID,,,
1,2,5,0
2,3,1,5
3,0,2,5
4,5,0,3


In [22]:
df = pd.DataFrame(data= {'A':['foo', 'foo', 'foo', 'foo', 'foo', 'bar', 'bar', 'bar', 'bar'],
                         'B':['one', 'one', 'one', 'two', 'two', 'one', 'one', 'two', 'tow'],
                         'C':['small', 'large', 'large', 'small', 'small', 'large', 'small', 'small', 'large'],
                         'D':[1, 2, 2, 3, 3, 4, 5, 6, 7],
                         'E':[2, 4, 5, 5, 6, 6, 8, 9, 9]})
print(df)

     A    B      C  D  E
0  foo  one  small  1  2
1  foo  one  large  2  4
2  foo  one  large  2  5
3  foo  two  small  3  5
4  foo  two  small  3  6
5  bar  one  large  4  6
6  bar  one  small  5  8
7  bar  two  small  6  9
8  bar  tow  large  7  9


In [23]:
table = pd.pivot_table(df,values=['D','E'], index=['A','C'])
table

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

In [24]:
table = pd.pivot_table(df,values=['D','E'], index=['A','C'], aggfunc={'D':np.mean, 'E':[min, max, np.mean]})
table

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

In [26]:
a = np.array([5,8,9,2])

In [27]:
np.min(a) # 최소값 리턴

2

In [28]:
np.max(a) # 최대값 리턴

9

In [29]:
np.argmin(a) # 최소값이 있는 인덱스 리턴

3

stack
- unstack은 index를 column으로 만들어준다.  
- stack은 column을 index으로 만들어준다.  
https://www.w3resource.com/pandas/dataframe/dataframe-stack.php

In [30]:
df_single_level_no_index = pd.DataFrame([[0,2],[3,4]], columns=['weight','height'])
df_single_level_no_index

,weight,height
0,0,2
1,3,4


In [32]:
df_single_level_no_index.stack()

0  weight    0
   height    2
1  weight    3
   height    4
dtype: int64

In [33]:
df_single_level_no_index.stack().unstack()

,weight,height
0,0,2
1,3,4


In [36]:
df_single_level_cols = pd.DataFrame([[0,2],[3,4]], index=['deer','monkey'], columns=['weight','height'])
df_single_level_cols

,weight,height
deer,0,2
monkey,3,4


In [37]:
df_single_level_cols.stack()

deer    weight    0
        height    2
monkey  weight    3
        height    4
dtype: int64

In [38]:
multicol1 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('weight', 'pounds')])
df_multi_level_cols1 = pd.DataFrame([[3, 4], [4, 5]],
                                    index=['deer', 'monkey'],
                                    columns=multicol1)
df_multi_level_cols1

weight       
           kg pounds
deer        3      4
monkey      4      5

In [39]:
df_multi_level_cols1.stack()

weight
deer   kg           3
       pounds       4
monkey kg           4
       pounds       5

In [40]:
df_multi_level_cols1.stack(1)

weight
deer   kg           3
       pounds       4
monkey kg           4
       pounds       5

In [41]:
df_multi_level_cols1.stack(0)

,,kg,pounds
deer,weight,3,4
monkey,weight,4,5


In [45]:
df_multi_level_cols1.stack(0).unstack()

,kg,pounds
,weight,weight
deer,3,4
monkey,4,5


In [44]:
swap = df_multi_level_cols1.stack(0).unstack()

#column의 계층(level) 순서변경
swap.swaplevel(0,1, axis=1) # 순서 바꾸기... 열이 바뀌니까 1... 열의 레벨이 바뀐다. 

weight       
           kg pounds
deer        3      4
monkey      4      5